In [ ]:
!python -c "import monai" || pip install -q "monai-weekly[gdown, nibabel, tqdm, ignite]"
!python -c "import matplotlib" || pip install -q matplotlib
%matplotlib inline

In [2]:
import os
import torch
from monai.networks.nets import UNet
from monai.transforms import (
    LoadImaged,
    EnsureChannelFirstd,
    Orientationd,
    Spacingd,
    ScaleIntensityRanged,
    CropForegroundd,
    Compose,
)
from monai.data import Dataset, DataLoader

# Use the provided directory as root_dir
root_dir = "/tmp/tmp8oke_gi_"

print("Root directory:", root_dir)

# Define your model architecture
device = torch.device("cuda:0")
model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=2,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
    norm="batch",
).to(device)

# Load the trained model weights
model.load_state_dict(torch.load(os.path.join(root_dir, "best_metric_model.pth")))
model.eval()

# Define example input
random_input = torch.randn(1, 1, 96, 96, 96).to(device)  # Adjust the shape according to your input

# Trace the model
traced_model = torch.jit.trace(model, random_input)

# Save the traced model to ONNX format
onnx_file_path = os.path.join(root_dir, "spleen_seg_model.onnx")
torch.onnx.export(
    traced_model,
    example_input,
    onnx_file_path,
    export_params=True,
    opset_version=11,
    do_constant_folding=True,
)

print("Traced model converted to ONNX successfully.")

print("ONNX file saved at:", onnx_file_path)


Root directory: /tmp/tmpy4so1vf5


/opt/app-root/lib64/python3.9/site-packages/torch/jit/_trace.py:1001: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 836 / 1769472 (0.0%)
Greatest absolute difference: 8.803606033325195e-05 at index (0, 1, 67, 31, 79) (up to 1e-05 allowed)
Greatest relative difference: 0.04067104894233465 at index (0, 0, 67, 31, 82) (up to 1e-05 allowed)
  _check_trace(


Traced model converted to ONNX successfully.
ONNX file saved at: /tmp/tmpy4so1vf5/spleen_seg_model.onnx
